In [1]:
import os
from pathlib import Path
import numpy as np
import json
from loguru import logger
import platform
import re 
import nibabel as nib
import statistics
import sys

import mri_data
import monai_training

from reload_recursive import reload_recursive

In [2]:
reload_recursive(mri_data)
reload_recursive(monai_training)
from mri_data.file_manager import scan_3Tpioneer_bids, DataSet, Scan  # noqa: E402, F401
from mri_data import utils
from mri_data import file_manager as fm
from monai_training import preprocess  # noqa: E402
from monai_training.preprocess import DataSetProcesser

In [3]:
logger.remove()
logger.add(sys.stderr, level="INFO")

1

In [4]:
#! Set these variables
work_dir_name = "choroid_pineal2_pituitary_T1-1"

In [5]:
# hostname = platform.node()
# if hostname == "rhinocampus":
#     drive_root = Path("/media/smbshare")
# else:
#     drive_root = Path("/mnt/h")
drive_root = fm.get_drive_root()

projects_root = Path("/home/srs-9/Projects")

msmri_home = projects_root / "ms_mri"
training_work_dirs = msmri_home / "training_work_dirs"

# dataroot = "/media/hemondlab/Data/3Tpioneer_bids"
dataroot = drive_root / "3Tpioneer_bids"
work_dir = training_work_dirs / work_dir_name
ensemble_out_dir = work_dir / "ensemble_output"

Scan the ensemble_out folder produced during the monai training to collect the predicted labels into a DataSet struct. Create another DataSet object with the same subjects and sessions but rooted in the main data location, then collect the manual labels 

In [6]:
labels = ["choroid_t1_flair", "pineal", "pituitary"]
ensemble_out_dataset = scan_3Tpioneer_bids(ensemble_out_dir, label="t1_ensemble.nii.gz")
dataset = DataSet.dataset_like(ensemble_out_dataset)
dataset.dataroot = dataroot
dataset_proc = DataSetProcesser(dataset)
dataset_proc.prepare_labels(["choroid_t1_flair", "pineal", "pituitary"], suffix_list=["CH", "SRS", "ED", "DT"])
dataset = dataset_proc.dataset
dataset.sort()
ensemble_out_dataset.sort()

2025-03-10 13:19:18.841 | INFO     | monai_training.preprocess:prepare_labels:176 - Prepare Labels
  0%|          | 0/10 [00:00<?, ?it/s]2025-03-10 13:19:18.889 | INFO     | mri_data.utils:combine_labels:89 - Combining labels for Scan(subid=2146, sesid=20191017)
2025-03-10 13:19:18.896 | SUCCESS  | monai_training.preprocess:log:22 - Saved /media/smbshare/3Tpioneer_bids/sub-ms2146/ses-20191017/choroid_t1_flair-ED.pineal-SRS.pituitary-CH.nii.gz
2025-03-10 13:19:18.899 | INFO     | mri_data.utils:combine_labels:89 - Combining labels for Scan(subid=1033, sesid=20171117)
2025-03-10 13:19:18.905 | SUCCESS  | monai_training.preprocess:log:22 - Saved /media/smbshare/3Tpioneer_bids/sub-ms1033/ses-20171117/choroid_t1_flair-CH.pineal-CH.pituitary-CH.nii.gz
2025-03-10 13:19:18.908 | INFO     | mri_data.utils:combine_labels:89 - Combining labels for Scan(subid=2081, sesid=20170204)
2025-03-10 13:19:18.914 | SUCCESS  | monai_training.preprocess:log:22 - Saved /media/smbshare/3Tpioneer_bids/sub-ms208

Calculate Dice Score for choroid

In [8]:
dice_scores = []
print("Dice Scores")
for scan, inference in zip(dataset, ensemble_out_dataset):
    man_label = fm.find_label(scan, "choroid_t1_flair", ["CH", "ED", "DT"])
    seg1 = nib.load(man_label).get_fdata()
    seg2 = nib.load(inference.label_path).get_fdata()

    # Calculate Dice score
    score = utils.dice_score(seg1, seg2, seg2_val=1)
    dice_scores.append(score)
    print("{}: {:0.2f}".format(scan.subid, score))
print("----------")
print("Mean: {:0.2f}".format(sum(dice_scores) / len(dice_scores)))
print("Std: {:0.2f}".format(statistics.stdev(dice_scores)))

print(dice_scores)

Dice Scores
1010: 0.73
1033: 0.77
1119: 0.67
1259: 0.60
1437: 0.67
1547: 0.74
2081: 0.76
2126: 0.76
2146: 0.58
2187: 0.73
----------
Mean: 0.70
Std: 0.07
[0.7307096374341494, 0.77152, 0.6692445794779727, 0.5991278094599127, 0.6666666666666666, 0.7399245283018868, 0.7567567567567568, 0.758714658794141, 0.5823765786452354, 0.7304015296367112]


Calculate Hausdorff Distance for Choroid

In [14]:
haus = []
print("Hausdorff Dist")
for scan, inference in zip(dataset, ensemble_out_dataset):
    man_label = fm.find_label(scan, "choroid_t1_flair", ["CH", "ED", "DT"])
    seg1 = nib.load(man_label).get_fdata()
    seg2 = nib.load(inference.label_path).get_fdata()

    # Calculate Dice score
    score = utils.hausdorff_dist(seg1, seg2, seg2_val=1)
    haus.append(score)
    print("{}: {:0.2f}".format(scan.subid, score))
print("----------")
print("Mean: {:0.2f}".format(sum(haus) / len(haus)))
print("Std: {:0.2f}".format(statistics.stdev(haus)))

print(haus)

Hausdorff Dist
1010: 58.81
1033: 1.41
1119: 2.00
1259: 2.00
1437: 3.16
1547: 3.16
2081: 1.41
2126: 2.21
2146: 2.45
2187: 1.41
----------
Mean: 7.80
Std: 17.93
[58.808932367209096, 1.4142135623730951, 2.0, 2.0, 3.1622776601683795, 3.1622776601683795, 1.4142135623730951, 2.212461179749725, 2.449489742783178, 1.4142135623730951]


Calculate Dice Score for pineal

In [7]:
dice_scores = []
print("Dice Scores")
for scan, inference in zip(dataset, ensemble_out_dataset):
    label = fm.find_label(scan, "pineal", ["SRS_T1", "CH", "SRS"])
    seg1 = nib.load(label).get_fdata()
    seg2 = nib.load(inference.label_path).get_fdata()

    # Calculate Dice score
    score = utils.dice_score(seg1, seg2, seg2_val=2)
    dice_scores.append(score)
    print("{}: {:0.2f}".format(scan.subid, score))
print("----------")
print("Mean: {:0.2f}".format(sum(dice_scores) / len(dice_scores)))
print("Std: {:0.2f}".format(statistics.stdev(dice_scores)))

print(dice_scores)

Dice Scores
1010: 0.84
1033: 0.82
1119: 0.59
1259: 0.82
1437: 0.78
1547: 0.87
2081: 0.79
2126: 0.56
2146: 0.00
2187: 0.68
----------
Mean: 0.68
Std: 0.26
[0.8387096774193549, 0.8177613320999075, 0.5931558935361216, 0.8221859706362153, 0.7848101265822784, 0.8699106256206555, 0.7857142857142857, 0.5615384615384615, 0.0, 0.6772908366533864]


Calculate Hausdorff Dist for pineal

In [16]:
haus = []
print("haus")
for scan, inference in zip(dataset, ensemble_out_dataset):
    label = fm.find_label(scan, "pineal", ["CH", "SRS"])
    seg1 = nib.load(label).get_fdata()
    seg2 = nib.load(inference.label_path).get_fdata()

    # Calculate Dice score
    score = utils.hausdorff_dist(seg1, seg2, seg2_val=2)
    haus.append(score)
    print("{}: {:0.2f}".format(scan.subid, score))
print("----------")
print("Mean: {:0.2f}".format(sum(haus) / len(haus)))
print("Std: {:0.2f}".format(statistics.stdev(haus)))

print(haus)

haus
1010: 135.92
1033: 1.41
1119: 2.00
1259: 1.00
1437: 1.00
1547: 3.32
2081: 1.00
2126: 1.41
2146: 15.67
2187: 1.41
----------
Mean: 16.41
Std: 42.23
[135.9154148726332, 1.4142135623730951, 2.0, 1.0, 1.0, 3.3166247903554, 1.0, 1.4142135623730951, 15.665240362042365, 1.4142135623730951]


Calculate DICE score for pituitary

In [16]:
dice_scores = []
print("Dice Scores")
for scan, inference in zip(dataset, ensemble_out_dataset):
    label = fm.find_label(scan, "pituitary", ["CH", "ED", "DT", "SRS"])
    seg1 = nib.load(label).get_fdata()
    seg2 = nib.load(inference.label_path).get_fdata()

    # Calculate Dice score
    score = utils.dice_score(seg1, seg2, seg2_val=3)
    dice_scores.append(score)
    print("{}: {:0.2f}".format(scan.subid, score))
print("----------")
print("Mean: {:0.2f}".format(sum(dice_scores) / len(dice_scores)))
print("Std: {:0.2f}".format(statistics.stdev(dice_scores)))

print(dice_scores)

Dice Scores
1010: 0.78
1033: 0.80
1119: 0.83
1259: 0.87
1437: 0.89
1547: 0.85
2081: 0.83
2126: 0.87
2146: 0.83
2187: 0.84
----------
Mean: 0.84
Std: 0.03
[0.7754991905018889, 0.7983320697498104, 0.832258064516129, 0.8744152572867938, 0.8856947296372347, 0.8515171078114913, 0.8250876509544215, 0.8696997833488084, 0.8270732614354779, 0.8442115292996665]


Calculate Hausdorff Dist for pituitary

In [11]:
haus = []
print("haus")
for scan, inference in zip(dataset, ensemble_out_dataset):
    label = fm.find_label(scan, "pituitary", ["CH", "SRS", "ED", "DT"])
    seg1 = nib.load(label).get_fdata()
    seg2 = nib.load(inference.label_path).get_fdata()

    # Calculate Dice score
    score = utils.hausdorff_dist(seg1, seg2, seg2_val=3)
    haus.append(score)
    print("{}: {:0.2f}".format(scan.subid, score))
print("----------")
print("Mean: {:0.2f}".format(sum(haus) / len(haus)))
print("Std: {:0.2f}".format(statistics.stdev(haus)))

print(haus)

haus
1010: 2.00
1033: 1.73
1119: 1.73
1259: 1.00
1437: 1.00
1547: 1.41
2081: 1.73
2126: 2.00
2146: 3.00
2187: 1.41
----------
Mean: 1.70
Std: 0.58
[2.0, 1.7320508075688772, 1.7320508075688772, 1.0, 1.0, 1.4142135623730951, 1.7320508075688772, 2.0, 3.0, 1.4142135623730951]


Save a new version of the predicted label that has a labelling index of 2 so that it appears as a different color in itksnap

In [ ]:
inference: Scan = ensemble_out_dataset[0]
label_path = inference.label_path
new_label_path = label_path.with_name(f"{fm.nifti_name(label_path)}_val2.nii.gz")
utils.set_label_value(label_path, new_label_path, 2)

PosixPath('/home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1010/ses-20180208/flair.t1_ensemble_val2.nii.gz')

In [11]:
for scan, inference in zip(dataset, ensemble_out_dataset):
    flair = scan.root / "flair.nii.gz"
    t1 = scan.root / "t1.nii.gz"
    images = [flair, t1]
    manual = scan.label_path
    prediction = inference.label_path
    labels = [manual]
    cmd = utils.open_itksnap_workspace_cmd(images, labels, win=True)
    print(cmd)

itksnap -g H:/3Tpioneer_bids/sub-ms1010/ses-20180208/flair.nii.gz -o H:/3Tpioneer_bids/sub-ms1010/ses-20180208/t1.nii.gz -s H:/3Tpioneer_bids/sub-ms1010/ses-20180208/choroid_t1_flair-CH.pineal-CH.pituitary-CH.nii.gz
itksnap -g H:/3Tpioneer_bids/sub-ms1033/ses-20171117/flair.nii.gz -o H:/3Tpioneer_bids/sub-ms1033/ses-20171117/t1.nii.gz -s H:/3Tpioneer_bids/sub-ms1033/ses-20171117/choroid_t1_flair-CH.pineal-CH.pituitary-CH.nii.gz
itksnap -g H:/3Tpioneer_bids/sub-ms1119/ses-20161010/flair.nii.gz -o H:/3Tpioneer_bids/sub-ms1119/ses-20161010/t1.nii.gz -s H:/3Tpioneer_bids/sub-ms1119/ses-20161010/choroid_t1_flair-CH.pineal-CH.pituitary-CH.nii.gz
itksnap -g H:/3Tpioneer_bids/sub-ms1259/ses-20200803/flair.nii.gz -o H:/3Tpioneer_bids/sub-ms1259/ses-20200803/t1.nii.gz -s H:/3Tpioneer_bids/sub-ms1259/ses-20200803/choroid_t1_flair-ED.pineal-SRS.pituitary-CH.nii.gz
itksnap -g H:/3Tpioneer_bids/sub-ms1437/ses-20210503/flair.nii.gz -o H:/3Tpioneer_bids/sub-ms1437/ses-20210503/t1.nii.gz -s H:/3Tpionee

Produce commands to open images and labels in freeview

In [1]:
image_paths = ["one", "twp", "three"]
step1 = " -g ".join(image_paths)
step1.split(" ")

['one', '-g', 'twp', '-g', 'three']

In [ ]:
for scan, inference in zip(dataset, ensemble_out_dataset):
    flair = scan.root / "flair.nii.gz"
    t1 = scan.root / "t1.nii.gz"
    manual = scan.label_path
    prediction = inference.label_path
    print("freeview", flair, t1, f"{prediction}:colormap=heat", f"{manual}:colormap=heat")

freeview /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/flair.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/t1.nii.gz /home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1010/ses-20180208/flair.t1_ensemble.nii.gz:colormap=heat /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/pineal-CH.nii.gz:colormap=heat
freeview /media/smbshare/3Tpioneer_bids/sub-ms1029/ses-20170816/flair.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1029/ses-20170816/t1.nii.gz /home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1029/ses-20170816/flair.t1_ensemble.nii.gz:colormap=heat /media/smbshare/3Tpioneer_bids/sub-ms1029/ses-20170816/pineal-SRS.nii.gz:colormap=heat
freeview /media/smbshare/3Tpioneer_bids/sub-ms1188/ses-20200720/flair.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1188/ses-20200720/t1.nii.gz /home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1188/ses-20200720/flair.t1_ensemble.nii.gz:colormap=heat